In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 22 09:09:54 2022

@author: faiza
"""

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_auc_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import category_encoders as ce
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb
import lightgbm as lgbm
import lime
from lime import lime_tabular
import shap
from IPython.display import display
from pdpbox import pdp, get_dataset, info_plots
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')


master_df = pd.read_csv('Master_File.CSV')

data = master_df
data = data.dropna(axis=1)
data.drop_duplicates(data.columns, keep='last')

data = data.drop(['pkSeqID','seq','stime','ltime','saddr','daddr','flgs','proto','state'], axis =1)


for index in range(len(data)):
    if isinstance(data.at[index,"sport"], str):
       data.at[index,"sport"] = -1

for index in range(len(data)):
    if isinstance(data.at[index,"dport"], str):
       data.at[index,"dport"] = -1

Theft_dataset = data[(data["category"] == "Theft") | (data["category"] == "Normal")]
Theft_dataset = Theft_dataset.drop(['subcategory'],axis = 1)

Data_Exfiltration_dataset = data[(data["subcategory"] == "Data_Exfiltration") | (data["subcategory"] == "Normal")]
Data_Exfiltration_dataset = Data_Exfiltration_dataset.drop(['category'],axis = 1)

Keylogging_dataset = data[(data["subcategory"] == "Keylogging") | (data["subcategory"] == "Normal")]
Keylogging_dataset = Keylogging_dataset.drop(['category'],axis = 1)

Theft_dataset.to_csv('Theft_File.CSV',index=False)

Data_Exfiltration_dataset.to_csv('Data_Exfiltration_File.CSV',index=False)

Keylogging_dataset.to_csv('Keylogging_File.CSV',index=False)



In [2]:
Theft_dataset = pd.read_csv('Theft_File.CSV')
target_features = Theft_dataset['attack']
#c_names = np.unique(Theft_dataset['category'])

In [3]:
#Normalize
scaler = MinMaxScaler()
Theft_dataset = Theft_dataset.drop(['attack','category'],axis=1)
f_names=Theft_dataset.columns

In [4]:
#RUS
print(Counter(target_features))
undersample = RandomUnderSampler(sampling_strategy=0.33, random_state=2)
x_over, y_over = undersample.fit_resample(Theft_dataset, target_features)
print(Counter(y_over))
Theft_dataset = x_over
target_features = y_over

Counter({0: 477, 1: 79})
Counter({0: 239, 1: 79})


In [5]:
#scaler.fit(Theft_dataset)
#Theft_dataset = scaler.transform(Theft_dataset)
#feature_selector = VarianceThreshold()
#VarianceFiltered_X = feature_selector.fit_transform(Theft_dataset)
time_dataset = Theft_dataset[['TnP_PerProto','proto_number']]
x_train, x_test, y_train, y_test = train_test_split(time_dataset,target_features,test_size=0.2, random_state=2)

In [6]:
%%timeit -r 1 -n 1

#NB
#NB = GaussianNB()
#model_NaiveBayes = NB.fit(x_train, y_train) 
#pred_nb = model_NaiveBayes.predict(x_test)
#print(classification_report(y_test, pred_nb))
#print("NB: ",roc_auc_score(y_test, pred_nb))

#LR
#LR = LogisticRegression()
#LR.fit(x_train, y_train)
#y_pred = LR.predict(x_test)
#print("LR: ",roc_auc_score(y_test, y_pred))

#LGBM
#lgbmc = lgbm.LGBMClassifier()
#lgbmc.fit(x_train, y_train)
#y_pred = lgbmc.predict(x_test)
#print("LGBM: ",roc_auc_score(y_test, y_pred))
#print(classification_report(y_test,y_pred))

#XGBC
xgbc = xgb.XGBClassifier()
xgbc.fit(x_train, y_train)
y_pred = xgbc.predict(x_test)
#print("XGBC: ",roc_auc_score(y_test, y_pred))
print(classification_report(y_test,y_pred))

#predictions = [round(value) for value in y_pred]
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

[11:57:32] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        55
           1       1.00      0.89      0.94         9

    accuracy                           0.98        64
   macro avg       0.99      0.94      0.97        64
weighted avg       0.98      0.98      0.98        64

38 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
#data["subcategory"].value_counts()

# Local explanation index
lexi = 3;
# Features to focus
ftf1 = 'TnP_PerProto'
ftf2 = 'AR_P_Proto_P_Dport'

# LIME
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(x_train),
    feature_names=f_names,
    mode='classification'
)

exp = explainer.explain_instance(
    data_row=x_test.iloc[lexi], 
    predict_fn=LR.predict_proba
)

exp.show_in_notebook(show_table=True)


# SHAP
shap_explainer = shap.TreeExplainer(lgbmc,x_test)
shap_values = shap_explainer.shap_values(x_test)
#shap_values = shap_explainer(x_test)
shap.initjs()
#shap.summary_plot(shap_values, x_test.values, plot_type="bar", class_names= c_names, feature_names = x_test.columns)

shap.summary_plot(shap_values, x_test.values, feature_names = x_test.columns , plot_type='violin')
#shap.summary_plot(shap_values, plot_type='violin')

shap.force_plot(shap_explainer.expected_value, shap_values[lexi],features =x_test.iloc[lexi,:] ,feature_names =x_test.columns)

#shap.plots.waterfall(shap_values[lexi])

shap.bar_plot(shap_values[lexi],features =x_test.iloc[lexi,:] ,feature_names =x_test.columns)

shap.dependence_plot(ftf1 , shap_values, x_test)

pd.set_option("display.max_rows", None, "display.max_columns", None)
x_test

y_test